# Model selection

In [1]:
from predict_model import clint_scorer, trump_scorer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import sys
import os
sys.path.append('../src/models')

In [2]:
with open('../data/processed/predictors.pickle', 'rb') as file:
    X = pickle.load(file)
with open('../data/processed/target.pickle', 'rb') as file:
    y = pickle.load(file)

In [3]:
clinton_metric = make_scorer(clint_scorer)
trump_metric = make_scorer(trump_scorer)

In [10]:
lr = LogisticRegression(
    C=2, penalty='l2', solver='lbfgs', multi_class='auto', max_iter=1000)

In [13]:
clinton_lr_scores = cross_val_score(lr, X, y, cv=5, scoring=clinton_metric, n_jobs=-1)
clinton_lr_scores

array([92.7, 89.1, 86.9, 90.3, 89.3])

In [29]:
round(clinton_rf_scores.mean(), 5)

89.42

In [14]:
trump_lr_scores = cross_val_score(lr, X, y, cv=5, scoring=trump_metric, n_jobs=-1)
trump_lr_scores

array([93.2, 83.6, 88.6, 85.3, 86.9])

In [30]:
round(trump_lr_scores.mean(), 5)

87.52

In [16]:
rf = RandomForestClassifier(n_estimators=100)

In [17]:
clinton_rf_scores = cross_val_score(rf, X, y, cv=5, scoring=clinton_metric, n_jobs=-1)
clinton_rf_scores

array([93.9, 90.1, 86.5, 88.4, 88.2])

In [28]:
print(round(clinton_rf_scores.mean(), 5))

89.42


In [21]:
trump_rf_scores = cross_val_score(rf, X, y, cv=5, scoring=trump_metric, n_jobs=-1)
trump_rf_scores

array([93.2, 85.3, 88.8, 88.4, 87.8])

In [26]:
print(round(trump_rf_scores.mean(),5))

88.7


In [31]:
knn = KNeighborsClassifier()

In [32]:
clinton_knn_scores = cross_val_score(knn, X, y, cv=5, scoring=clinton_metric, n_jobs=-1)
clinton_knn_scores

array([92.5, 88.9, 87. , 86.5, 86.9])

In [35]:
round(clinton_knn_scores.mean(), 2)

88.36

In [33]:
trump_knn_scores = cross_val_score(knn, X, y, cv=5, scoring=trump_metric, n_jobs=-1)
trump_knn_scores

array([88.9, 78. , 82.2, 82.2, 82.6])

In [36]:
round(trump_knn_scores.mean(), 2)

82.78

In [37]:
svm = SVC(kernel='linear')

In [38]:
clinton_svm_scores = cross_val_score(svm, X, y, cv=5, scoring=clinton_metric, n_jobs=-1)
clinton_knn_scores

array([92.5, 88.9, 87. , 86.5, 86.9])

In [39]:
trump_svm_scores = cross_val_score(svm, X, y, cv=5, scoring=trump_metric, n_jobs=-1)
trump_svm_scores

array([92.8, 83.6, 87.9, 85.3, 86.3])

## K-means clustering

# Parameter tuning

gridsearchcv

# Test 

Which model does better with test data? 
Stick with one metric
Whatever model is best, variation between train and test,etc. 